In [213]:
from collections import defaultdict,Counter
import random
from decimal import Decimal, getcontext

In [222]:
"""
@author: KhangTran2503
"""
class Arithmetic:
    def __init__(self):
        self.data = ''
        self.table_prob = defaultdict(list)
        self.terminator = '$'
        self.encode = 0
    
    def __get_table(self):
            count = Counter(self.data).items()
            total = sum(map(lambda x: x[1],count))
            
            low_prob = 0
            for key, val in count:
                prob = val/total
                self.table_prob[key].append(prob)
                self.table_prob[key].append((low_prob,low_prob + prob))
                low_prob += prob
            
    @staticmethod        
    def float2bin(number):
        whole, des = str(number).split(".")
        res = bin(int(whole))[2:] + '.'
        des = number - int(whole)
        res = ''
        while (des != int(des)):
            des = des * 2
            res += str(int(des) % 2)
            if int(des) >= 2:
                des -= 2
        return res

    @staticmethod
    def bin2float(bin_code):
        bin1, bin2 = bin_code.split('.')
        res = 0.0
        
        bin1 =  bin1[::-1]
        pw2 = 1
        for b in bin1:
            res += int(b)*pw2
            pw2 *= 2
        
        pw2sub = 0.5
        for b in bin2:
            res += int(b)*pw2sub
            pw2sub /= 2
        return res
        
    def compress(self,data):
        # assign value
        self.data = data + self.terminator
        self.__get_table()
#         print(self.table_prob)
#         compresss
        Low = 0.0
        High = 1.0
        Range = 1.0
        for symbol in self.data:
            (Low_prob, High_prob) = self.table_prob[symbol][1]
            new_Low = Low + Range*Low_prob
            new_High = Low + Range*High_prob
            Low, High, Range = new_Low, new_High, new_High - new_Low
            #print(Low,High)
        
        value = random.uniform(Low,High)
        #print(Low,High,value)
        #print(value)
        self.encode = self.float2bin(value)
    
    def share_encode(self):
        return (self.encode,self.table_prob)
            
    @classmethod
    def decompress(cls,data_encode):
        (encode,table_prob) = data_encode

        value = cls.bin2float('0.'+ encode)
        Low = 0
        High = 1.0
        Range = 1.0
        res = ''
        symbol = ''
        Terminator = '$'
        while 1:
            #print(round(value,25))
            for key in table_prob:
                (key_Low,key_High) = table_prob[key][1]
                if key_Low <= value < key_High:
                    symbol = key
                    break
            if symbol == Terminator: break
            Low = key_Low
            High = key_High
            Range = High - Low
            res += symbol
            value = (value - Low) / Range
        return res

            
            

# 1. Encode

In [226]:
encoder = Arithmetic()
encoder.compress('CAEE')

In [235]:
encoder = Arithmetic()
encoder.compress('1 2 3 4 5 6 7 8 9')

# 2. Decode

**Hạn chế:** Những xâu vượt quá độ dài 17 thì decompress không còn chính xác nữa.

**Nguyên Nhân:** Do phần thập phân trong Python còn hạn chế

In [236]:
data_encode = encoder.share_encode()
#print(data_encode)

In [237]:
print(Arithmetic().decompress(data_encode))

1 2 3 4 5 6 7 8 9
